In [ ]:
import torch
import numpy as np
import torch.nn as nn

embed_size = 4 # each word is represented with length 4 vector
my_embedding = nn.Embedding(10, embed_size) # 10 words are supported. each word is mapped on either number 0 - 9

input = torch.IntTensor([[1,2,4,5,5,5],
                         [4,3,0,9,9,9]]) # two sentences with sequence length of 6
                                                        # elements shoudl be integer
embedded_input = my_embedding(input)
print(embedded_input.size())
print("embedding for 4th token (5) is:", embedded_input[0, 3, :].element())
print("embedding for 5th token (5) is:", embedded_input[0, 4, :].element())

In [2]:
input = torch.LongTensor([[1,2,4,5,6,7],
                         [4,3,0,9,10,11]]) # does not work as we cover up to 10
embedded_input = my_embedding(input)

IndexError: index out of range in self

In [3]:
from torch.autograd import Variable

class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)   # d_model: vector length of input 
        position = torch.arange(0, max_len).unsqueeze(1) # arrange: return 1D tensor from a to b-1
        #print(position.size())  # torch.Size([5000, 1])
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        #print(div_term)
        pe[:, 0::2] = torch.sin(position * div_term) # ::2 means every other
        pe[:, 1::2] = torch.cos(position * div_term)
        #print(pe.size())      # torch.Size([5000, 4])
        pe = pe.unsqueeze(0)  
        #print(pe.size())      # torch.Size([1, 5000, 4]) <- the first dim will be # of sentences in input
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], requires_grad=False)
        return self.dropout(x)

In [6]:
import math
      
pe = PositionalEncoding(embed_size, 0)  # d_model: vector length of input (12 based on previous cells)
#pe_pure = pe.forward(Variable(torch.zeros(batch_size,10, embed_size))) # send zero vector to see the pure positional encoding part
embedded_input_w_pe = pe.forward(embedded_input)

In [7]:
print(embedded_input)
print(embedded_input_w_pe)

tensor([[[-5.7126e-01,  2.8596e-01,  6.1990e-01,  4.9004e-02],
         [-1.5105e+00,  4.4296e-01, -1.2973e+00, -3.7072e-01],
         [ 9.0008e-01, -1.5065e-03, -1.8334e+00, -4.3780e-01],
         [-7.3055e-01, -5.2499e-01, -1.3492e+00, -1.3128e+00],
         [-7.3055e-01, -5.2499e-01, -1.3492e+00, -1.3128e+00],
         [-7.3055e-01, -5.2499e-01, -1.3492e+00, -1.3128e+00]],

        [[ 9.0008e-01, -1.5065e-03, -1.8334e+00, -4.3780e-01],
         [-4.8804e-01, -1.0614e-01, -1.1204e+00,  1.2158e+00],
         [ 1.0974e+00, -5.8310e-01, -9.4776e-01,  2.4841e-01],
         [-4.6786e-02, -8.3433e-01, -8.2454e-01, -1.6718e+00],
         [-4.6786e-02, -8.3433e-01, -8.2454e-01, -1.6718e+00],
         [-4.6786e-02, -8.3433e-01, -8.2454e-01, -1.6718e+00]]],
       grad_fn=<EmbeddingBackward0>)
tensor([[[-0.5713,  1.2860,  0.6199,  1.0490],
         [-0.6690,  0.9833, -1.2873,  0.6292],
         [ 1.8094, -0.4177, -1.8134,  0.5620],
         [-0.5894, -1.5150, -1.3192, -0.3133],
         [-1.48

In [ ]:
q[3,:]